In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import pandas as pd
import numpy.random as rnd
from scipy.integrate import odeint,quad
from scipy.stats import kde,beta
import seaborn as sns
%matplotlib inline
from importlib import reload
pi=np.pi


from numpy import linalg as LA
from scipy.linalg import expm
from scipy.optimize import brentq


In [3]:
import stablecompoper
sns.set()
plt.rc('text', usetex=False)

## The monotype linear birth and death process in a periodic environment

This process $(X(t),t\ge 0)$ with values in $\mathbb{N}$ is described by its
time varying generator
\begin{equation}
  L_t f(x) = x\left[\lambda(t)(f(x+1)-f(x)) + \mu(t) (f(x-1)-f(x))\right]\,,
\end{equation}
where $\lambda,\mu$ are non negative $T$-periodic functions.

Let $Z(t)$ be the  point measure on $S$ describing the set of states
(i.e. phases) of the individuals born before $t$ and still alive at
time $t$ : if $Z(t) = \sum_i \delta_{s_i}$ then $<Z(t), f>=
\sum_i f(s_i)$. We have the convergence in $L^1$, when we start with $X(s)=1$ one individual of phase $s$,
\begin{equation}
  \lim_{n\to +\infty} e^{-\alpha(nT-s)} <Z(nT),f> = h(s) \int_S f(t)\, d\pi(t)\,,
\end{equation}
where the reproductive value of phase $s$ is the periodic function for $T=1$
\begin{equation}
  h(s) = e^{\alpha s -\varphi(s)}\,,
\end{equation}
and the measure $\pi$ is the stable composition law
\begin{equation}
  \boxed{\pi(dt) = \frac1{e^{A(T)} -1} \lambda(t) e^{A(t)}\, 1_{t\in(0,T)}\, dt\,.}
\end{equation}


The process is one dimensional, and the death rate is constant
$\mu(t)=\mu_0$ and the birth rate is
\begin{equation}
  \lambda(t) = \lambda_0 (1 + c \cos(2\pi t/T))\,.
\end{equation}
The stable composition law is thus
\begin{equation}
\pi(dt) = \frac1{e^{A(T)} -1} \lambda(t) e^{A(t)}\, 1_{t\in(0,T)}\,
dt\,,
\end{equation}
with
\begin{equation}
  A(t) = \lambda_0 (t + \frac{ c T}{2 \pi} \sin(2\pi t/T) )
\end{equation}
We perform a simulation of the linear birth and death process for $N$
periods, and we keep the phase, the birth dates modulo $T$, of the
living individuals at time $N T$. We wait until the first non extinct
population, and then we plot its histogram against the true density
$\pi$ and against the birth rate $\lambda(t)$

In [4]:
#reload(stablecompoper)
#from ipywidgets import interact, interactive, fixed, interact_manual
#import ipywidgets as widgets
#interact(stablecompoper.nsestimdenszchi,
#         lzero=widgets.FloatSlider(min=0.0, max=4.0, step=0.1, value=0.8, continuous_update=False),
#                muzero=widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=0.1, continuous_update=False),
#                  T=widgets.IntSlider(min=1, max=10, step=1, value=2, continuous_update=False),
#N=widgets.IntSlider(min=1, max=20, step=1, value=8, continuous_update=False),
#         coeff=widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=0.5, continuous_update=False),
#                 )


interactive(children=(FloatSlider(value=0.8, continuous_update=False, description='lzero', max=4.0), FloatSlid…

<function stablecompoper.nsestimdenszchi(lzero, muzero, T, N, coeff=1.0, estimnoyau=False, image=False)>

In [5]:
#reload(stablecompoper)
#from IPython.display import display
#w=interactive(stablecompoper.nsestimdenszchi,
#         lzero=widgets.FloatSlider(min=0.0, max=4.0, step=0.1, value=0.8, continuous_update=False),
#                muzero=widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=0.1, continuous_update=False),
#                  T=widgets.IntSlider(min=1, max=10, step=1, value=2, continuous_update=False),
#N=widgets.IntSlider(min=1, max=20, step=1, value=8, continuous_update=False),
#         coeff=widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=0.5, continuous_update=False),
#                 )


In [6]:
#display(w)

interactive(children=(FloatSlider(value=0.8, continuous_update=False, description='lzero', max=4.0), FloatSlid…

#### Remarque
Il ne faut pas partir avec un nombre de périodes $N$ trop grand, ni une période $T$ trop grande, sinon la taille de l'échantillon est beaucoup trop grande. Pour avoir une bonne estimation de la densité par l'histogramme, il suffit d'avoir une taile d'échantillon au dessus de 2000.En considerant le cas constant, on voit qu'il faut à peu près prendre $e^{N T (\lambda_0 -\mu_0)}\simeq 2000$ ce qui donne $NT \simeq 7.6/(\lambda_0 -\mu_0) \simeq 12$

Pour obtenir le cas où les taux de naissance et de mort sont constants, 
i l suffit de prendre c=0.

In [7]:
from ipywidgets import GridspecLayout,Layout,Button, AppLayout,TwoByTwoLayout,interactive_output
import ipywidgets as widgets
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto'))
grid = GridspecLayout(3, 3)
blzero=widgets.FloatSlider(min=0.0, max=4.0, step=0.1, value=0.8, continuous_update=False,description=r'$\lambda_0$')
bmuzero=widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=0.1, continuous_update=False,description=r'$\mu_0$')
bT=widgets.IntSlider(min=1, max=10, step=1, value=2, continuous_update=False,description='T')
bN=widgets.IntSlider(min=1, max=20, step=1, value=8, continuous_update=False,description='N')
bcoeff=widgets.FloatSlider(min=0.0, max=2.0, step=0.1, value=0.5, continuous_update=False,description='c')

grid[0,0]=blzero
grid[0,1]=bmuzero
grid[0,2]=bcoeff
grid[1,0]=bT
grid[1,1]=bN
w=interactive_output(stablecompoper.nsestimdenszchi,{'lzero':blzero,'muzero':bmuzero,'T':bT,'N':bN,'coeff':bcoeff})
display(grid,w)
#grid


GridspecLayout(children=(FloatSlider(value=0.8, continuous_update=False, description='$\\lambda_0$', layout=La…

Output()